In [1]:
import pandas as pd
import sqlalchemy as sa

In [2]:
engine = sa.create_engine('mssql+pyodbc://LAPTOP-QF2CI5Q7\SQLEXPRESS/PortfolioProject?driver=SQL+Server+Native+Client+11.0')

In [3]:
%load_ext sql

In [4]:
%sql mssql+pyodbc://LAPTOP-QF2CI5Q7\SQLEXPRESS/PortfolioProject?driver=SQL+Server+Native+Client+11.0

Let's overview the total cases and total deaths of the coronavirus by countries

In [5]:
%%sql
select location, sum(new_cases) as Total_Cases, sum(cast(new_deaths as int)) as Total_Deaths, population
from PortfolioProject..CovidDeaths 
where location in ('Canada','United States','United Kingdom','France','Germany','Russia')
group by location, population
order by 1

 * mssql+pyodbc://LAPTOP-QF2CI5Q7\SQLEXPRESS/PortfolioProject?driver=SQL+Server+Native+Client+11.0
Done.


location,Total_Cases,Total_Deaths,population
Canada,1422148.0,26245,37742157.0
France,6184552.0,111230,67564251.0
Germany,3735399.0,90883,83783945.0
Russia,5428961.0,132314,145934460.0
United Kingdom,4791628.0,128390,67886004.0
United States,33652097.0,604474,331002647.0


Shows what percentage of population got covid in Canada and what percentage of it died

In [6]:
%%sql
select location, population, sum(new_cases) as Total_Cases, sum(cast(new_deaths as int)) as Total_Deaths, 
(sum(new_cases)/population)*100 as CovidPercentage, (sum(cast(new_deaths as int))/sum(new_cases))*100 as DeathPercentage
from PortfolioProject..CovidDeaths
where location = 'Canada' group by location, population

 * mssql+pyodbc://LAPTOP-QF2CI5Q7\SQLEXPRESS/PortfolioProject?driver=SQL+Server+Native+Client+11.0
Done.


location,population,Total_Cases,Total_Deaths,CovidPercentage,DeathPercentage
Canada,37742157.0,1422148.0,26245,3.768062328817084,1.84544787181081


Looking at 10 Countries that have more than 20 million population and Highest Infection Rate 

In [7]:
%%sql
select top 10 location, population, max(total_cases) as HighestInfectionCount, max(total_cases/population)*100 as HighestCovidPercentage
from PortfolioProject..CovidDeaths
where population>20000000
Group by location, population
order by HighestCovidPercentage desc

 * mssql+pyodbc://LAPTOP-QF2CI5Q7\SQLEXPRESS/PortfolioProject?driver=SQL+Server+Native+Client+11.0
Done.


location,population,HighestInfectionCount,HighestCovidPercentage
United States,331002647.0,33652098.0,10.16671567584171
Argentina,45195777.0,4447701.0,9.84096589378251
France,67564251.0,5978650.0,8.848836346901855
Brazil,212559409.0,18513305.0,8.709708540824932
Colombia,50882884.0,4213074.0,8.279943408868098
Spain,46754783.0,3799733.0,8.126939654494814
Poland,37846605.0,2879811.0,7.609166000490665
South America,430759772.0,32772980.0,7.608180273621279
European Union,444919060.0,32717352.0,7.353551452706926
United Kingdom,67886004.0,4791628.0,7.058344456391924


Looking at 10 Countries With Highest Death Count per Population

In [8]:
%%sql
select top 10 location, max(cast(total_deaths as int)) as TotalDeathCount
from PortfolioProject..CovidDeaths
--where location = 'Canada'
where continent is not null
Group by location
order by TotalDeathCount desc

 * mssql+pyodbc://LAPTOP-QF2CI5Q7\SQLEXPRESS/PortfolioProject?driver=SQL+Server+Native+Client+11.0
Done.


location,TotalDeathCount
United States,604474
Brazil,515985
India,397637
Mexico,232803
Peru,191899
Russia,132314
United Kingdom,128390
Italy,127542
France,111230
Colombia,105934


Showing the Continent with Highest Death Count per Population

In [9]:
%%sql
select continent, max(cast(total_deaths as int)) as TotalDeathCount
from PortfolioProject..CovidDeaths
--where location = 'Canada'
where continent is not null
Group by continent
order by TotalDeathCount desc

 * mssql+pyodbc://LAPTOP-QF2CI5Q7\SQLEXPRESS/PortfolioProject?driver=SQL+Server+Native+Client+11.0
Done.


continent,TotalDeathCount
North America,604474
South America,515985
Asia,397637
Europe,132314
Africa,60264
Oceania,910


GLOBAL NUMBERS OF TOTAL CASES, TOTAL DEATHS, TOTAL CASE PERCENTAGE AND TOTAL DEATH PERCENTAGE

In [75]:
%%sql
select sum(new_cases) as total_cases, SUM(cast(new_deaths as int)) as total_deaths,
convert(Decimal(10,2),(sum(new_cases)/7900000000)*100) as TotalCasePercentage,
convert(Decimal(10,2),SUM(cast(new_deaths as int))/SUM(new_cases)*100) as DeathPercentage
from PortfolioProject..CovidDeaths
--where continent is not null
--group by location
order by 1,2

 * mssql+pyodbc://LAPTOP-QF2CI5Q7\SQLEXPRESS/PortfolioProject?driver=SQL+Server+Native+Client+11.0
Done.


total_cases,total_deaths,TotalCasePercentage,DeathPercentage
576933446.0,12550205,7.30,2.18


Total Number of Tested People and Positive Rates by Location

In [80]:
%%sql
select location, sum(cast(new_tests as int)) as Total_tests, CONVERT(DECIMAL(10,2),avg(cast(positive_rate as float))*100) as Positive_rate
from PortfolioProject..CovidVaccinatons
where location in ('Canada','United States','United Kingdom','France','Germany','Russia')
group by location
order by 1


 * mssql+pyodbc://LAPTOP-QF2CI5Q7\SQLEXPRESS/PortfolioProject?driver=SQL+Server+Native+Client+11.0
Done.


location,Total_tests,Positive_rate
Canada,36641854,3.68
France,92536448,5.40
Germany,None,5.75
Russia,134819067,3.32
United Kingdom,197309938,3.88
United States,466195371,7.45


Number of Vaccinated People and Percentage in Canada

In [12]:
%%sql
select location,max(cast(people_vaccinated as int)) - max(cast(people_fully_vaccinated as int)) as One_dose, 
max(cast(people_fully_vaccinated as int)) as Two_doses, max(cast(people_vaccinated_per_hundred as float)) as one_dose_percentage,
max(cast(people_fully_vaccinated_per_hundred as float)) as fully_vaccinated_percentage
from PortfolioProject..CovidVaccinatons 
where location = 'Canada'
group by location
order by 1

 * mssql+pyodbc://LAPTOP-QF2CI5Q7\SQLEXPRESS/PortfolioProject?driver=SQL+Server+Native+Client+11.0
Done.


location,One_dose,Two_doses,one_dose_percentage,fully_vaccinated_percentage
Canada,15021039,10589254,67.86,28.06
